### Embedding

In [ ]:
import boto3
import json
bedrock = boto3.client(
            service_name='bedrock-runtime',
            region_name='us-west-2')
prompt = 'canberra is capital of australia'
body = json.dumps({
    "inputText": prompt,
})

model_id = 'amazon.titan-embed-text-v1'
accept = 'application/json'
content_type = 'application/json'

response = bedrock.invoke_model(
                    body=body,
                    modelId = model_id,
                    accept=accept,
                    contentType=content_type)

# print (response)
response_body = json.loads(response['body'].read())
embedding = response_body.get('embedding')
print (embedding)

In [ ]:
from text_to_sql_agent import TextToSQLAgent

agent = TextToSQLAgent()

In [ ]:
user_prompt = "what are the customer emails"
result = agent.process_prompt(user_prompt)
print(result)


In [ ]:
user_prompt = "provide me the sql only"
result = agent.process_prompt(user_prompt)
print(result)


In [ ]:
user_prompt = "some all the ids then minus the sells price"
result = agent.process_prompt(user_prompt)
print(result)


In [ ]:
user_prompt = "what was my first question"
result = agent.process_prompt(user_prompt)
print(result)


In [ ]:
user_prompt = "generate me a pandas dataframe sample table"
result = agent.process_prompt(user_prompt)
print(result)


In [ ]:
user_prompt = "select the second column"
result = agent.process_prompt(user_prompt)
print(result)


In [ ]:
user_prompt = "what is the value of the 3rd column, second value"
result = agent.process_prompt(user_prompt)
print(result)


In [ ]:
user_prompt = "rename the columsn to city related"
result = agent.process_prompt(user_prompt)
print(result)


In [ ]:
user_prompt = "tell me a story about dragon"
result = agent.process_prompt(user_prompt)
print(result)


In [ ]:
user_prompt = "tell me more about it"
result = agent.process_prompt(user_prompt)
print(result)


In [ ]:
from TAI.genai import AWSBedrock
ab = AWSBedrock()
for i in ab.get_active_models():
    print (i)
#     print (i['modelId'])